[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cornflake15/twitter_sentiment_analysis/blob/master/tweepy-vader.ipynb)

# Twitter Sentiment Analysis

In [1]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
CONSUMER_API_KEY = '8fxjhgCnLZie0AjbLuntKT0EM'
CONSUMER_API_SECRET = 'd9oab3yrWQybhC64hk5YIXybhuIql20sUWvte2Z8vVQvKtM3Id'
ACCESS_TOKEN ='822833106-VKB22OHa8dXm2Hd5sEl5d5gWLmSfaH5EEmjA88DX'
ACCESS_TOKEN_SECRET = '22iR0cQfOvSQ77UFKKf4CHNLPvvNYEqzNUkugvubrLMx5'
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Utility Function

In [3]:
# Function to get trending topic by country name
def get_trending_topic_by_country(country, num_trends):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    trends_country = api.trends_available()
    for trend in trends_country:
        if trend['name'] == country:
            country_info = (trend['name'], trend['woeid'])
            print('Country: {}, WOEID: {}'.format(trend['name'], trend['woeid']))
    
    index = 1
    trending_topics = []
    trends_result = api.trends_place(country_info[1])
    for tren in trends_result[0]['trends'][:num_trends]: 
        print('#{}: {}'.format(index, tren['name']))
        trending_topics.append(tren['name'])
        index += 1
        
    return trending_topics

# Function to get tweets by keyword
def get_tweets_by_keyword(keywords, num_of_tweets, language, filename='', export_to_csv=False):
    """
    Use this function if you want export the tweet to CSV file
    keywords: tweet keyword
    num_of_tweets: number of tweets that your want to fetch
    language: tweet language
    filename: filename
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
    
    
    if export_to_csv == True:
        export_to_csv(df_tweets, filename)
        
    return df_tweets

# Overiding function 
def get_tweets_by_keyword(keywords, num_of_tweets, language):
    """
    Use this function and its parameter if you don't want export to CSV
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
        
    return df_tweets

# Function to remove unwanted string in the tweet
def remove_unwanted_string(dataframe):
    new_record = []
    tweet_text = dataframe.apply(lambda x: str(x).split())
    for record in tweet_text:
        for text in record:
            if 'RT' in text or '@' in text or '#' in text:
                record.remove(text)
        
        new_record.append(' '.join(record))
    dataframe = new_record
    return dataframe
            
# Function to print sentiments of the sentence
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object
    sid_obj = SentimentIntensityAnalyzer()
    
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary
    # which contains pos, neg, neu, and compoind scores
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict
    
# Function export to CSV
def export_to_csv(dataframe, filename):
    print('\nExported to CSV')
    return dataframe.to_csv(filename)

## Get Tweets by trending topics on spesific country

In [4]:
df_tweets_trending = get_trending_topic_by_country('Indonesia', 10)

Country: Indonesia, WOEID: 23424846
#1: Johnny
#2: #InstitutPalingBorjuis
#3: #globallivefmxbrightwin
#4: #PKS_SarangKadrun
#5: S.Pd
#6: Hendri
#7: #YOUNGJAEXDINGO
#8: #WeekendSamaShopee
#9: Klasifikasi
#10: jhonny


## Get Tweets by Keyword

We can pass trending topic or any keyword as a parameter to the function.

In [5]:
df_tweets = get_tweets_by_keyword(['corona', 'COVID-19'], 10, 'en')

In [6]:
df_tweets.head()

username      screen_name  \
0                                 Susie Swanson   SusanMSwanson1   
1                            Naresh Kumar Singh           AdvBsr   
2  Jammu Kashmir & Ladakh Police Fans Club 🇮🇳 🍁           jkpsfc   
3                                Salumu Mtaturu   mtaturu_salumu   
4                                   Subham Paul  SubhamP84280622   

                    user_location  user_verified  followers_count  \
0                              NY          False              136   
1  Race Corse Colony Buland Shahr          False              694   
2       Jammu And Kashmir, Bharat          False            11572   
3                Mwanza, Tanzania          False              192   
4                                          False                0   

   friends_count  listed_count    tweet_created_at  \
0            292             0 2020-06-20 11:11:11   
1            165             4 2020-06-20 11:10:59   
2           8186            45 2020-06-20 11:10:28   
3            213             0 2020-06-20 11:10:27   
4              2             0 2020-06-20 11:10:08   

                                          tweet_text tweet_coordinates  \
0  RT @ScottGo18320060: COVID-19 Hoax - The Crime...              None   
1  RT @firstpost: L-G Anil Baijal ordered a 5-day...              None   
2  RT @Dhama_IPS: In our effort to contain Corona...              None   
3  RT @brayson_lema: Being of service is by itsel...              None   
4  RT @f9hiMNCLFeo39HS: @ImranKhanPTI Containment...              None   

  tweet_place  tweet_favorite_count  
0        None                     0  
1        None                     0  
2        None                     0  
3        None                     0  
4        None                     0

In [7]:
df_tweets['tweet_text'][1]

'RT @firstpost: L-G Anil Baijal ordered a 5-day mandatory institutional-quarantine for every #COVID19 patient under home-isolation in #Delhi…'

## Remove Unwanted String from Tweets

In [8]:
df_tweets['tweet_text'] = remove_unwanted_string(df_tweets['tweet_text'])
print(df_tweets['tweet_text'])

0    @ScottGo18320060: COVID-19 Hoax - The Crime of...
1    @firstpost: L-G Anil Baijal ordered a 5-day ma...
2    @Dhama_IPS: In our effort to contain Corona st...
3    @brayson_lema: Being of service is by itself a...
4    @f9hiMNCLFeo39HS: Containment of the new crisi...
5    @hemantrajora_: Big Story : In AIIMS Delhi, 13...
6    @Dhama_IPS: In our effort to contain Corona st...
7    @hemantrajora_: Big Story : In AIIMS Delhi, 13...
8    Homemade healthy vegetable juice recipe. Read ...
9    @ARSHAD7649: If you are too close, you can bre...
Name: tweet_text, dtype: object


## Tweet Sentiment with VaderSentiment library/module

Note: Can not support Indonesian language

In [9]:
pos, neg, neu, comp = [], [], [], []
for text in df_tweets['tweet_text']:
    sentiment = sentiment_scores(text)
    pos.append(sentiment['pos'])
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    comp.append(sentiment['compound'])
    
df_tweets_sentiment = pd.DataFrame()
# df_tweets_sentiment.columns = ['tweet_text', 'positive', 'negative', 'neutral', 'compound']
df_tweets_sentiment['tweet_text'] = df_tweets['tweet_text']
df_tweets_sentiment['positive'] = pos
df_tweets_sentiment['negative'] = neg
df_tweets_sentiment['neutral'] = neu
df_tweets_sentiment['compound'] = comp

In [10]:
df_tweets_sentiment

tweet_text  positive  negative  \
0  @ScottGo18320060: COVID-19 Hoax - The Crime of...     0.101     0.176   
1  @firstpost: L-G Anil Baijal ordered a 5-day ma...     0.085     0.000   
2  @Dhama_IPS: In our effort to contain Corona st...     0.000     0.000   
3  @brayson_lema: Being of service is by itself a...     0.123     0.083   
4  @f9hiMNCLFeo39HS: Containment of the new crisi...     0.000     0.177   
5  @hemantrajora_: Big Story : In AIIMS Delhi, 13...     0.153     0.000   
6  @Dhama_IPS: In our effort to contain Corona st...     0.000     0.000   
7  @hemantrajora_: Big Story : In AIIMS Delhi, 13...     0.153     0.000   
8  Homemade healthy vegetable juice recipe. Read ...     0.213     0.000   
9  @ARSHAD7649: If you are too close, you can bre...     0.000     0.000   

   neutral  compound  
0    0.723   -0.3400  
1    0.915    0.0772  
2    1.000    0.0000  
3    0.795    0.2960  
4    0.823   -0.6249  
5    0.847    0.5574  
6    1.000    0.0000  
7    0.847    0.5574  
8    0.787    0.4019  
9    1.000    0.0000

Detail information about the output visit https://github.com/cjhutto/vaderSentiment